# DEVWKS-2192: Automate your network with Nornir – Python automation framework!

### Exploring inventory

In [ ]:
from pprint import pprint
from colorama import Fore
import time

from nornir import InitNornir
nr = InitNornir(config_file="config.yaml")
pprint(nr.inventory.hosts)

In [ ]:
pprint(nr.inventory.groups)

### Simple output collection with netmiko

In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

results = nr.run(task=netmiko_send_command, command_string="show ip int brief | ex una")
print_result(results)

### Simple data retrieval using napalm

In [ ]:
from nornir.plugins.tasks.networking import napalm_get
results = nr.run(
    task=napalm_get, getters=["facts", "interfaces"]
)
print_result(results)

### Exploring connections

In [ ]:
for host in nr.inventory.hosts.values():
    print(f"{host.name} connections: {host.connections}")
    
nr.close_connections()
print(f"{Fore.RED}All connections have been closed{Fore.RESET}", end="\n\n")

for host in nr.inventory.hosts.values():
    print(f"{host.name} connections: {host.connections}")

### Data retrieval

In [ ]:
r1 = nr.inventory.hosts['R1']
print(r1.data)

In [ ]:
print(r1['tags'])  # directly from data
print(r1['ntp'])  # from group
print(r1['snmp_community'])  # from defaults
print(r1.get('non-existent-key', 'Placeholder'))  # this key does not exist in any group

### Change data dynamically

In [ ]:
# Settings site and locator for every host
for host in nr.inventory.hosts.values():
    site = host.groups[0]
    host.data['site'] = site
    locator = f'{host.name}.{site}'
    host.data['locator'] = locator

r1 = nr.inventory.hosts['R1']
print(f"{r1.name} has the following data: {r1.data}")
print(f"{r1.name} site: {r1['site']}, locator: {r1['locator']}")

### Filtering

In [ ]:
print(list(nr.filter(locator="R1.Barcelona").inventory.hosts.keys()))
print(list(nr.filter(site="Barcelona").inventory.hosts.keys()))

### Advanced filtering
#### Filter function


In [ ]:
print(list(nr.filter(filter_func=lambda h: h.has_parent_group('London')).inventory.hosts.keys()))

#### F filter

In [ ]:
from nornir.core.filter import F

print(list(nr.filter(F(locator="R1.Barcelona")).inventory.hosts.keys()))
print(list(nr.filter(F(groups__contains="London")).inventory.hosts.keys()))
print(list(nr.filter(F(groups__contains="London") & F(tags__contains="isr4400")).inventory.hosts.keys()))
print(list(nr.filter(F(groups__contains="London") & F(tags__all=["isr4400", "edge"])).inventory.hosts.keys()))
print(list(nr.filter(F(ntp__servers__contains="1.2.3.4")).inventory.hosts.keys()))

### Combining filtering and task execution


In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

london_devices = nr.filter(F(groups__contains="London"))
result = london_devices.run(task=netmiko_send_command, command_string="show ip route")
print_result(result)

### Custom tasks

In [ ]:
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import netmiko_send_command

def get_commands(task, commands):
    for command in commands:
        task.run(task=netmiko_send_command, command_string=command)
        
london_devices = nr.filter(F(groups__contains="London"))
result = london_devices.run(task=get_commands, commands=["show ip int br", "show arp"])
print_result(result)


#### Result, MultiResult, AggregatedResult
* Result — result from a single task
* MultiResult – list of results from grouped tasks
* AggregatedResult – dictionary where the key is device name and the value is MultiResult

In [ ]:
pprint(result)
pprint(result['R10'])
print()
print(repr(result['R10'][1]), end='\n\n')
print(result['R10'][1].result)

In [ ]:
nr.close_connections()

## END